# Notebook 3: Diagnostics & Sensitivity Analysis

Validates the causal assumptions behind the estimates from Notebook 02:
covariate balance, propensity score overlap, Rosenbaum bounds,
E-values, and falsification tests.

In [1]:
import sys
sys.path.insert(0, "..")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from src.preprocessing import load_dataset, clean_data, engineer_features
from src.causal_models import PropensityScoreMatching, IPWEstimator
from src.diagnostics.balance import balance_table, love_plot, assess_overlap
from src.diagnostics.sensitivity import rosenbaum_bounds, compute_e_value, sensitivity_plot
from src.diagnostics.placebo import placebo_treatment_test, negative_control_test
from src.utils.config import TREATMENT_COL, OUTCOME_HEALTH, COVARIATE_COLS, RANDOM_SEED
from src.utils.visualization import plot_propensity_distribution

/home/akshay/.local/lib/python3.12/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


## Load & Prepare Data

In [2]:
try:
    df = pd.read_csv("../data/processed/cleaned_data.csv")
except FileNotFoundError:
    df = engineer_features(clean_data(load_dataset()))

covs = [c for c in COVARIATE_COLS if c in df.columns]
X = df[covs].values
T = df[TREATMENT_COL].values
Y = df[OUTCOME_HEALTH].values
print(f"n={len(df)}, covariates={covs}")

n=10000, covariates=['age', 'female', 'race_hispanic', 'race_other', 'race_white', 'education', 'income', 'state_id', 'urban']


---
## 1. Covariate Balance Before Adjustment

Standardised Mean Difference (SMD) > 0.1 indicates meaningful imbalance.

In [3]:
bal_raw = balance_table(df, covs, treatment_col=TREATMENT_COL)
print(bal_raw.to_string(index=False))

imbalanced = bal_raw[bal_raw["smd_unadjusted"].abs() > 0.1]
print(f"\nImbalanced covariates (|SMD|>0.1): {len(imbalanced)}/{len(covs)}")

    covariate  smd_unadjusted
          age          0.2679
       female         -0.1942
race_hispanic         -0.0163
   race_other          0.0464
   race_white         -0.0958
    education         -0.2757
       income          0.0256
     state_id          0.0261
        urban          0.0460

Imbalanced covariates (|SMD|>0.1): 3/9


---
## 2. Propensity Score Overlap

In [4]:
psm = PropensityScoreMatching(seed=RANDOM_SEED)
psm.fit(X, T)
ps = psm.propensity_scores_

overlap = assess_overlap(ps, T)
for k, v in overlap.items():
    print(f"  {k}: {v}")

plot_propensity_distribution(ps, T, save=True, filename="overlap_diagnostics.png")
plt.show()

  treated_ps_mean: 0.21488572357659821
  control_ps_mean: 0.18094901890184664
  treated_ps_range: (0.05886513941506564, 0.5077304383587632)
  control_ps_range: (0.05083323042316092, 0.4852183452390144)
  positivity_violation_rate: 0.0


---
## 3. Covariate Balance After PSM

In [5]:
psm.match(T)
df_matched = psm.get_matched_data(df, treatment_col=TREATMENT_COL)
bal_matched = balance_table(df_matched, covs, treatment_col=TREATMENT_COL)

# Combine for Love plot
bal_compare = bal_raw.copy()
bal_compare["smd_adjusted"] = bal_matched["smd_unadjusted"].values
print(bal_compare.to_string(index=False))

love_plot(bal_compare, save=True, filename="love_plot_psm.png")
plt.show()

    covariate  smd_unadjusted  smd_adjusted
          age          0.2679       -0.0304
       female         -0.1942        0.0119
race_hispanic         -0.0163       -0.0063
   race_other          0.0464        0.0080
   race_white         -0.0958       -0.0269
    education         -0.2757       -0.0208
       income          0.0256        0.0143
     state_id          0.0261       -0.0460
        urban          0.0460       -0.0277


---
## 4. Covariate Balance After IPW

In [6]:
ipw = IPWEstimator(seed=RANDOM_SEED)
ipw.fit(X, T)

bal_ipw = balance_table(df, covs, treatment_col=TREATMENT_COL, weights=ipw.weights_)
print(bal_ipw.to_string(index=False))

love_plot(bal_ipw, save=True, filename="love_plot_ipw.png")
plt.show()

    covariate  smd_unadjusted  smd_adjusted
          age          0.2679       -0.0054
       female         -0.1942       -0.0030
race_hispanic         -0.0163       -0.0041
   race_other          0.0464        0.0016
   race_white         -0.0958        0.0043
    education         -0.2757       -0.0136
       income          0.0256       -0.0030
     state_id          0.0261        0.0010
        urban          0.0460        0.0042


---
## 5. Sensitivity Analysis: Rosenbaum Bounds

How strong would unmeasured confounding need to be (Gamma) to
explain away the result?

In [7]:
t_idx, c_idx = psm.matched_indices_
bounds = rosenbaum_bounds(Y[t_idx], Y[c_idx])

print(f"{'Gamma':>8}  {'Upper p':>10}  {'Sig (p<.05)':>12}")
print("-" * 34)
for b in bounds:
    sig = "Yes" if b["upper_p"] < 0.05 else "No"
    print(f"{b['gamma']:>8.2f}  {b['upper_p']:>10.4f}  {sig:>12}")

sensitivity_plot(bounds, save=True)
plt.show()

   Gamma     Upper p   Sig (p<.05)
----------------------------------
    1.00      0.0000           Yes
    1.25      0.0000           Yes
    1.50      0.0000           Yes
    1.75      0.0000           Yes
    2.00      0.0000           Yes
    2.25      0.0000           Yes
    2.50      0.0000           Yes
    2.75      0.0000           Yes
    3.00      0.0005           Yes


---
## 6. E-value Analysis

In [8]:
# Approximate risk ratio from matched-pair means
mean_t = Y[t_idx].mean()
mean_c = Y[c_idx].mean()
rr = mean_t / mean_c if mean_c != 0 else 1.0

ate_result = psm.estimate_ate(Y, T, n_bootstrap=100)
rr_ci = (mean_c + ate_result["ci_upper"]) / mean_c

ev = compute_e_value(rr, rr_ci)
print(f"E-value (point):     {ev['e_value_point']:.3f}")
print(f"E-value (CI bound):  {ev['e_value_ci']:.3f}")
print(f"\nAn unmeasured confounder would need RR >= {ev['e_value_point']:.2f}")
print("with both treatment and outcome to explain away the effect.")

E-value (point):     1.344
E-value (CI bound):  1.321

An unmeasured confounder would need RR >= 1.34
with both treatment and outcome to explain away the effect.


---
## 7. Falsification Tests

In [9]:
# Placebo treatment test
print("Running placebo test (50 permutations)...")
placebo = placebo_treatment_test(
    X, Y, T, PropensityScoreMatching,
    n_permutations=50, seed=RANDOM_SEED,
)
print(f"Real ATE:       {placebo['real_ate']:.4f}")
print(f"Placebo mean:   {placebo['placebo_mean']:.4f}")
print(f"Placebo std:    {placebo['placebo_std']:.4f}")
print(f"p-value:        {placebo['p_value']:.4f}")

Running placebo test (50 permutations)...


Real ATE:       -4.8890
Placebo mean:   -0.0061
Placebo std:    0.3132
p-value:        0.0000


In [10]:
# Negative control: smoking should not cause age
neg = negative_control_test(
    X, T, df["age"].values, PropensityScoreMatching, seed=RANDOM_SEED,
)
print(f"Smoking -> Age ATE: {neg['negative_control_ate']:.4f}")
print(f"CI: [{neg['ci_lower']:.4f}, {neg['ci_upper']:.4f}]")
print(f"Covers zero: {neg['covers_zero']}")

Smoking -> Age ATE: -0.5875
CI: [-0.9733, 1.2268]
Covers zero: True


## Summary

1. **Balance achieved** after both PSM and IPW adjustment.
2. **Overlap** is adequate — low positivity violation rate.
3. **Sensitivity:** results are moderately robust to unmeasured confounding.
4. **Falsification:** placebo test rejects the null; negative control shows null effect.

These diagnostics support a causal interpretation of the smoking → health effect.
**Next:** Notebook 04 consolidates results and presents policy conclusions.